In [111]:
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import tensorflow as tf
import sys
import random
from scipy import spatial

NOTE: Currently the model only handles `batch_size=1`. As there are only 221 professions for debiasing, this is a non-issue, but this should be changed in a future version to accomodate larger sets of words to debias.

In [112]:
vocab_size = 22000
vector_dim = 100
batch_size = 1
k_knn = 20


In [4]:
# wiki_model = Word2Vec.load("english-wikipedia-articles-20170820-models/enwiki_2017_08_20_fasttext.model")

In [5]:
wvs = wiki_model.wv.vectors[:vocab_size]

In [17]:
def cosine_similarity(a,b):
    return 1 - spatial.distance.cosine(a, b)

In [20]:
inapp_gen_indices = np.load('inapp_gen_indices.npy')
inapp_gen_indices

array([    0,     1,     2, ..., 21997, 21998, 21999])

See `debias_embedding_probabilistic.ipynb` for construction of gender_word_pairs

In [123]:
gender_word_pairs = np.load('gender_word_pairs.npy')


Construct the gender vector (see Bolukbasi)

In [125]:
g = np.mean([wiki_model.wv.get_vector(f) - wiki_model.wv.get_vector(m) for m,f in gender_word_pairs],axis=0)

(100,)

In [126]:
biases = np.array([cosine_similarity(wvs[i], g) for i in inapp_gen_indices])
biases.shape
#biases

(21872,)

In [128]:
female_neighbors = np.array([wiki_model.wv.index2word[ind] for ind in inapp_gen_indices[(biases.argsort()[::-1])[:500]]])


In [ ]:
# np.save('female_neighbors', female_neighbors)

In [130]:
male_neighbors = np.array([wiki_model.wv.index2word[ind] for ind in inapp_gen_indices[(biases.argsort())[:500]]])


In [131]:
male_neighbors

array(['warlord', 'abbot', 'chiefs', 'captain', 'sgt', 'simeon',
       'colonel', 'umpire', 'muhammad', 'godfrey', 'lord', 'sack', 'davy',
       'hitler', 'caesar', 'friar', 'brigadier', 'horde', 'sidekick',
       'joker', 'pope', 'captaincy', 'wrath', 'capt', 'ruler',
       'chieftain', 'judah', 'johnny', 'kirby', 'halfback', 'cromwell',
       'crook', 'bat', 'conqueror', 'assassin', 'abraham', 'hooker',
       'haig', 'bodyguard', 'wizard', 'napoleon', 'soldier', 'ibn',
       'oswald', 'ezekiel', 'henry', 'robber', 'vizier', 'knight', 'buck',
       'bishop', 'caliph', 'assassins', 'broncos', 'squad', 'goblin',
       'crusader', 'mccoy', 'joe', 'nobleman', 'batsman', 'darius',
       'sheriff', 'dick', 'chamberlain', 'barons', 'mlb', 'maj', 'jr',
       'boss', 'john', 'kicker', 'bud', 'emperor', 'slain', 'pardon',
       'hrer', 'duke', 'horseback', 'hank', 'gus', 'nfl', 'xi', 'mob',
       'mr', 'wyatt', 'kirk', 'keeper', 'apostle', 'budd', 'striker',
       'archbishop', 's

In [ ]:
# np.save('male_neighbors', male_neighbors)

Professions come from Bolukbasi paper 

https://github.com/tolga-b/debiaswe/blob/master/data/professions.json

In [132]:
professions = np.load('professions.npy')
professions_inds = np.load('professions_inds.npy')

In [133]:
len(professions)

221

Generate batch produces the index of `batch_size` random professions

In [25]:
# generate batch data
def generate_batch(batch_size):
    indicies_p = np.random.choice(professions_inds,1,replace=False)
    return indicies_p


In [26]:
indicies_p = generate_batch(batch_size)
indicies_p

array([11787])

In [27]:
wiki_model.wv.index2word[2235]

'historian'

In [28]:
original_weights = wiki_model.wv.vectors[:vocab_size,:]

In [29]:
bolukbasi_debiased_vecs = np.load('bolukbasi_debiased_vecs.npy')

In [32]:
male_neighbors = np.load('male_neighbors.npy')
male_neighbors_inds = np.array([wiki_model.wv.vocab[w].index for w in male_neighbors])


In [33]:
female_neighbors = np.load('female_neighbors.npy')
female_neighbors_inds = np.array([wiki_model.wv.vocab[w].index for w in female_neighbors])


In [34]:
neigbors = np.concatenate([female_neighbors,male_neighbors])


In [36]:
neighbors_inds = np.concatenate([female_neighbors_inds,male_neighbors_inds])


In [140]:
graph = tf.Graph()

with graph.as_default():
    with tf.name_scope('inputs'):
        indices_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    with tf.device('/cpu:0'):
        with tf.name_scope('embeddings'):
            # initialize the embedding weights
            embedding = tf.Variable(bolukbasi_debiased_vecs)
            # embed profession(s)
            embed_inputs = tf.nn.embedding_lookup(embedding,indices_inputs)
#             embed_neighbors = tf.nn.embedding_lookup(embedding,neighbors_inds)
            embed_neighbors_m = tf.nn.embedding_lookup(embedding,male_neighbors_inds)
            embed_neighbors_f = tf.nn.embedding_lookup(embedding,female_neighbors_inds)


            # L1 distance
#             distances_all = tf.reduce_sum(tf.abs(tf.add(embed_neighbors, tf.negative(embed_inputs))), reduction_indices=1)

            distances_m = tf.reduce_sum(tf.abs(tf.add(embed_neighbors_m, tf.negative(embed_inputs))), reduction_indices=1)

            distances_f = tf.reduce_sum(tf.abs(tf.add(embed_neighbors_f, tf.negative(embed_inputs))), reduction_indices=1)

            # L2 Distance
#             distance = tf.sqrt(tf.reduce_sum(tf.squared_difference(neighbors, profession), reduction_indices=1))
            
            # Get indicies of smallest distances (nearest neighbors)
#             _,inds_all = tf.math.top_k(tf.negative(distances_all),k=k_knn)

            vals_m,inds_m = tf.math.top_k(tf.negative(distances_m),k=(k_knn//2))

            vals_f,inds_f = tf.math.top_k(tf.negative(distances_f),k=(k_knn//2))
            
            # True if the neighbors are from the 'male' half of the neighbors
#             bools = tf.math.greater(inds_all,(len(neighbors_inds)//2))
            
            # fraction of neighbors that are 'male' biased words
#             proportion = tf.reduce_mean(tf.cast(bools, tf.float32))
            
    with tf.name_scope('loss'):
        loss = tf.reduce_sum(tf.abs(vals_m - vals_f))
    
    tf.summary.scalar('loss', loss)
    
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(0.003).minimize(loss)
    
#     norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
#     normalized_embeddings = embeddings / norm
    
    merged = tf.summary.merge_all()
    
    init = tf.global_variables_initializer()
#     saver = tf.train.Saver()     

In [141]:
indices_p = generate_batch(batch_size)

In [142]:
indices_p

array([8947])

In [143]:
wiki_model.wv.index2word[indices_p[0]]

'teenager'

In [144]:
feed_dict = {indices_inputs: indices_p}
with tf.Session(graph=graph) as sess:
    init.run()
    res = sess.run(loss, feed_dict)
    print(res)

5.4857163


In [ ]:
num_steps = 1000001

with tf.Session(graph=graph) as session:
    
    init.run()
    print('Initialized')
    min_loss = sys.maxsize
    average_loss = 0
    for step in range(num_steps):
        indices_wv = generate_batch(batch_size)
        feed_dict = {indices_inputs: indices_wv}

      # Define metadata variable.
        run_metadata = tf.RunMetadata()

        _, summary, loss_val = session.run([optimizer, merged, loss],
                                         feed_dict=feed_dict,
                                         run_metadata=run_metadata)
        average_loss += loss_val

        if step % 2000 == 0 and step != 0:
            if step > 0:
                average_loss /= 2000
        # The average loss is an estimate of the loss over the last 2000
        # batches.
            print('Average loss at step ', step, ': ', average_loss)
            if average_loss < min_loss:
                print('New min loss, saving embedding')
                wvs_embed_trained = embedding.eval()

                min_loss = average_loss
                
            average_loss = 0    

    # Save the model for checkpoints.
    saver.save(session, './model_0')
    

In [146]:
wvs_embed_trained = np.load('wvs_embed_trained_goldberg.npy')

In [147]:
np.save('fasttext_wiki_debias_cluster_wvs_weights', wvs_embed_trained)

In [148]:
wvs_embed_trained_norm = np.sqrt(np.sum(np.square(wvs_embed_trained), 1, keepdims=True))
wvs_embed_trained_normalized_embeddings = wvs_embed_trained / wvs_embed_trained_norm


In [149]:
np.save('fasttext_wiki_debias_cluster_wvs', wvs_embed_trained_normalized_embeddings)


In [150]:
def convert_np_to_bin_model(np_vectors,model_name):
    with open(model_name+'.txt', 'w') as we:
        we.write('{} {}\n'.format(vocab_size,vector_dim))
        for i in range(vocab_size):
            w = wiki_model.wv.index2word[i]
            vec = np_vectors[i]
            we.write('{} '.format(w))
            for v in vec:
                we.write(str(v) + ' ')
            we.write('\n')
    model = KeyedVectors.load_word2vec_format(model_name+'.txt', binary=False)
    model.save_word2vec_format(model_name+'.bin', binary=True)
    print('created model '+model_name+'.bin')

In [152]:
convert_np_to_bin_model(wvs_embed_trained_normalized_embeddings,'fasttext_wiki_debias_cluster')



created model fasttext_wiki_debias_cluster.bin
